
<H1>Introduction</H1>

A new Worldwide Pizza Chain wants to open their stores in Toronto . They have a Big Brand name , To start with they want to do the following

1) They want to open 5 Pizza stores in Toronto , they want to open stores in such a way that that they have coverage for entire city.

2) they want to divide the city into Four zones , Each zone will have one store and Pizza wil be delivered from that store to locations in that zone .

3) In each zone for opening the store 

a) If there are more than 20 venues at more than multiple locations then we look for top 3 places which have maximum pizza store. 

b) If there are less than 20 venues at all locations then we look for only 1 top places which have maximum pizza store

<h1> First we import the required Libraries </H1>

In [3]:
import requests
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries imported.


<H1> Getting Postal Code Data from WikiPedia Link</H1>

In [8]:
# Getting postalCode data from wikipedia link
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
My_table = soup.find('table',{'class':'wikitable sortable'})
labels=[]
data=[]
for tx in My_table.find_all('th'):
         labels.append(tx.text.strip())
rows = My_table.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    if cols:
     data.append([ele for ele in cols if ele])
    
df = pd.DataFrame.from_records(data, columns=labels)
df.rename(columns={'Postcode': 'PostalCode'}, inplace=True)
df.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


<H1> Removing Borough which are not assigned </H1>

In [10]:
df1=df[df.Borough != 'Not assigned']
df1.shape

(212, 3)

<H1> Updating rows where Neighbourhood is not assigned with name of Borough </H1>

In [12]:
for index, row in df1.iterrows():
     if (row['Neighbourhood'] == 'Not assigned'):
        row['Neighbourhood'] = row['Borough']

<H1>Reading co-ordinates from Geospatial_Coordinates.csv file for each postal code </H1>

In [14]:
import sys
import types
import pip

if not('ibm-cos-sdk' in [package.project_name for package in pip.get_installed_distributions()]):
    !pip install ibm-cos-sdk==2.0.0 -q

import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_63009217aad749368c287a2b914cbff9 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='GvA3fdd2BSZEgS-8td33u1F-z92vvb-T3lTnAKf-C-e5',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_63009217aad749368c287a2b914cbff9.get_object(Bucket='datavisual-donotdelete-pr-2gfq67enauernl',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

geodata = pd.read_csv(body)
geodata.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
result = pd.merge(df1,geodata,on="PostalCode")
result.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763


<H1>Performing clustering using K-MEAN and creating 5 clusters based on latitute and longitude to divide Toronto </H1>

In [16]:
# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library
from urllib.parse import urljoin
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np

df_grouped = result.groupby('Neighbourhood').mean().reset_index()

kclusters = 5

df_clustering = df_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

k_means_cluster_centers = kmeans.cluster_centers_

df_merged = df_clustering

# add clustering labels
df_merged['Cluster Labels'] = kmeans.labels_


<H1>Displaying on map each cluster with different colour</H1>

In [17]:
latitude=43.6542599
longitude=-79.3606359

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

#set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

#print (folium.__version__)


# add markers to the map
#markers_colors = []
for lat, lon, cluster in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<H1>Creating a dataset with PostalCode, neighbourhood and cluster number </H1>

In [19]:
df_merged['Neighbourhood'] = df_grouped['Neighbourhood']
toronto_clustered = pd.merge(result,df_merged,on=['Neighbourhood','Latitude','Longitude'])
toronto_clustered.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels
0,M3A,North York,Parkwoods,43.753259,-79.329656,3
1,M4A,North York,Victoria Village,43.725882,-79.315572,2
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636,0
4,M6A,North York,Lawrence Heights,43.718518,-79.464763,3


<H1> Calling FourSquare API to get location data for each venue </H1>

<H1>We will be taking venue count and using it to find neighbourhood with most venue </H1> 
<H1> If number of venue at multiple neighbourhood is more than 20 we pick top 3 neighbourhood with most pizza stores </H1>

In [21]:
CLIENT_ID = 'T100LAO1E4PPVOWA2HPGLEMG0W4DV22XNKGMHIDD0RKE0XOC' # your Foursquare ID
CLIENT_SECRET = 'IYD1RDNHDYF5LKFJDJSU54XMX2ECMIPO4KFD1KZEEZOJADZY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT=100

def getNearbyVenues(names,latitudes,longitudes, PostalCode,Borough,ClusterLabels,radius=500):
    
    venues_list=[]
    for name, lat, lng,pc,borough,cluster in zip(names,latitudes,longitudes,PostalCode,Borough,ClusterLabels):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            pc,
            borough,
            cluster,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood',
                  'PostalCode',
                  'Borough',
                  'ClusterLabels',
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [22]:

toronto_venues = getNearbyVenues(names=toronto_clustered['Neighbourhood'],
                                   latitudes=toronto_clustered['Latitude'],
                                   longitudes=toronto_clustered['Longitude'],
                                   PostalCode=toronto_clustered['PostalCode'],
                                   Borough=toronto_clustered['Borough'],
                                   ClusterLabels=toronto_clustered['Cluster Labels']
                                  )

Parkwoods
Victoria Village
Harbourfront
Regent Park
Lawrence Heights
Lawrence Manor
Queen's Park
Islington Avenue
Rouge
Malvern
Don Mills North
Woodbine Gardens
Parkview Hill
Ryerson
Garden District
Glencairn
Cloverdale
Islington
Martin Grove
Princess Gardens
West Deane Park
Highland Creek
Rouge Hill
Port Union
Flemingdon Park
Don Mills South
Woodbine Heights
Humewood-Cedarvale
Bloordale Gardens
Eringate
Markland Wood
Old Burnhamthorpe
Guildwood
Morningside
West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor
Downsview North
Wilson Heights
Thorncliffe Park
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Scarborough Village
Fairview
Henry Farm
Oriole
Northwood Park
York University
East Toronto
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
East Birchmount Park
Ionview
Kennedy Park
Bayview Village
CFB Toronto
Downsview East
The Danforth West
Riverdale
Design Exchange
Toronto

<H1>Get the venue count in each neighbourhood</H1>

In [23]:
toronto_venues.groupby(['Neighbourhood']).count()

,PostalCode,Borough,ClusterLabels,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,,,,
Adelaide,100,100,100,100,100,100,100,100,100
Agincourt,4,4,4,4,4,4,4,4,4
Agincourt North,3,3,3,3,3,3,3,3,3
Albion Gardens,9,9,9,9,9,9,9,9,9
Alderwood,11,11,11,11,11,11,11,11,11
Bathurst Manor,19,19,19,19,19,19,19,19,19
Bathurst Quay,14,14,14,14,14,14,14,14,14
Bayview Village,4,4,4,4,4,4,4,4,4
Beaumond Heights,9,9,9,9,9,9,9,9,9


<H1> Now we analyze Cluster0 locations </H1>

In [25]:
Toronto_Zone1=toronto_venues.loc[toronto_venues['ClusterLabels'] == 0].groupby(['Neighbourhood']).count().sort_values(['Venue'],ascending=False)
Toronto_Zone1.head(20)

,PostalCode,Borough,ClusterLabels,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,,,,
Adelaide,100,100,100,100,100,100,100,100,100
Design Exchange,100,100,100,100,100,100,100,100,100
Victoria Hotel,100,100,100,100,100,100,100,100,100
Union Station,100,100,100,100,100,100,100,100,100
Underground city,100,100,100,100,100,100,100,100,100
Toronto Islands,100,100,100,100,100,100,100,100,100
Toronto Dominion Centre,100,100,100,100,100,100,100,100,100
Ryerson,100,100,100,100,100,100,100,100,100
Richmond,100,100,100,100,100,100,100,100,100


<H1> Since in cluster 0 we have multiple locations having more than 20 venue , locations having maximum pizza places </H1>

In [26]:
Toronto_pizza_zone1=toronto_venues.loc[toronto_venues['ClusterLabels'] == 0]
Toronto_pizza_zone1=Toronto_pizza_zone1[Toronto_pizza_zone1['Venue Category'].str.contains("Pizza")]
Toronto_pizza_zone1.groupby(['Neighbourhood']).count().sort_values(['Venue'],ascending=False).head(5)

,PostalCode,Borough,ClusterLabels,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,,,,
Union Station,4,4,4,4,4,4,4,4,4
Toronto Islands,4,4,4,4,4,4,4,4,4
Harbourfront East,4,4,4,4,4,4,4,4,4
Davisville,3,3,3,3,3,3,3,3,3
Little Portugal,2,2,2,2,2,2,2,2,2


<H1> Three places identified in Cluster0  are 'Union Station' , 'Toronto Islands' ,'Harbourfront East' </H1>

<H1> Now we analyze Cluster1 locations </H1>

In [28]:
toronto_venues.loc[toronto_venues['ClusterLabels'] == 1].groupby(['Neighbourhood']).count().sort_values(['Venue'],ascending=False)

,PostalCode,Borough,ClusterLabels,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,,,,
Swansea,37,37,37,37,37,37,37,37,37
High Park,23,23,23,23,23,23,23,23,23
The Junction South,23,23,23,23,23,23,23,23,23
New Toronto,14,14,14,14,14,14,14,14,14
Humber Bay Shores,14,14,14,14,14,14,14,14,14
Mimico South,14,14,14,14,14,14,14,14,14
Roncesvalles,13,13,13,13,13,13,13,13,13
Parkdale,13,13,13,13,13,13,13,13,13
Kingsway Park South West,12,12,12,12,12,12,12,12,12


<H1> Since there are only 3 neighbourhood with more than 20 venues  , so all 3 neighbourhood are identified  </H1>

<H1>Three places identified in Cluster1 are 'Swansea' , 'High Park' ,'The Junction South'</H1>

<H1> Now we analyze cluster2 Locations </H1>

In [30]:
toronto_venues.loc[toronto_venues['ClusterLabels'] == 2].groupby(['Neighbourhood']).count().sort_values(['Venue'],ascending=False)

,PostalCode,Borough,ClusterLabels,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,,,,
Steeles West,17,17,17,17,17,17,17,17,17
L'Amoreaux West,17,17,17,17,17,17,17,17,17
Woodbine Gardens,13,13,13,13,13,13,13,13,13
Parkview Hill,13,13,13,13,13,13,13,13,13
Sullivan,10,10,10,10,10,10,10,10,10
Clarks Corners,10,10,10,10,10,10,10,10,10
Tam O'Shanter,10,10,10,10,10,10,10,10,10
Golden Mile,9,9,9,9,9,9,9,9,9
Clairlea,9,9,9,9,9,9,9,9,9


In [31]:
Toronto_Zone2=toronto_venues.loc[toronto_venues['ClusterLabels'] == 2]
Toronto_Zone2=Toronto_Zone2[Toronto_Zone2['Venue Category'].str.contains("Pizza")]
Toronto_Zone2.groupby(['Neighbourhood']).count().sort_values(['Venue'],ascending=False)

,PostalCode,Borough,ClusterLabels,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,,,,
Clarks Corners,2,2,2,2,2,2,2,2,2
Parkview Hill,2,2,2,2,2,2,2,2,2
Sullivan,2,2,2,2,2,2,2,2,2
Tam O'Shanter,2,2,2,2,2,2,2,2,2
Woodbine Gardens,2,2,2,2,2,2,2,2,2
Guildwood,1,1,1,1,1,1,1,1,1
L'Amoreaux West,1,1,1,1,1,1,1,1,1
Morningside,1,1,1,1,1,1,1,1,1
Steeles West,1,1,1,1,1,1,1,1,1


<H1> Places identified in Cluster2 is 'Steeles West / L'Amoreaux West' since it has 1 pizza place 
and in this cluster  since neighbourhood having maximum venues is less than 20 so we would choose only 1 neighbourhood out of the 2 </H1>

<H1> Now we analyze Cluster 3 </H1>

In [34]:
toronto_venues.loc[toronto_venues['ClusterLabels'] == 3].groupby(['Neighbourhood']).count().sort_values(['Venue'],ascending=False)

,PostalCode,Borough,ClusterLabels,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,,,,
Oriole,64,64,64,64,64,64,64,64,64
Fairview,64,64,64,64,64,64,64,64,64
Henry Farm,64,64,64,64,64,64,64,64,64
Willowdale South,33,33,33,33,33,33,33,33,33
Bedford Park,25,25,25,25,25,25,25,25,25
Lawrence Manor East,25,25,25,25,25,25,25,25,25
North Toronto West,23,23,23,23,23,23,23,23,23
Don Mills South,20,20,20,20,20,20,20,20,20
Flemingdon Park,20,20,20,20,20,20,20,20,20


<H1> Three places identified in Cluster3 are 'Fairview' , 'Oriole' ,'Henry Farm'  , since  these 3 neighbourhood have more than 20 venues  </H1>

<H1> Now we analyze Cluster4 </H1>

In [37]:
toronto_venues.loc[toronto_venues['ClusterLabels'] == 4].groupby(['Neighbourhood']).count().sort_values(['Venue'],ascending=False)

,PostalCode,Borough,ClusterLabels,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,,,,
Albion Gardens,9,9,9,9,9,9,9,9,9
Beaumond Heights,9,9,9,9,9,9,9,9,9
Thistletown,9,9,9,9,9,9,9,9,9
South Steeles,9,9,9,9,9,9,9,9,9
Silverstone,9,9,9,9,9,9,9,9,9
Humbergate,9,9,9,9,9,9,9,9,9
Mount Olive,9,9,9,9,9,9,9,9,9
Jamestown,9,9,9,9,9,9,9,9,9
Westmount,7,7,7,7,7,7,7,7,7


<H1> Number of Pizza Places in Cluster4 neighbourhoods </H1>

In [39]:
Toronto_Zone4=toronto_venues.loc[toronto_venues['ClusterLabels'] == 4]
Toronto_Zone4=Toronto_Zone4[Toronto_Zone4['Venue Category'].str.contains("Pizza")]
Toronto_Zone4.groupby(['Neighbourhood']).count().sort_values(['Venue'],ascending=False)

,PostalCode,Borough,ClusterLabels,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,,,,
Westmount,2,2,2,2,2,2,2,2,2
Albion Gardens,1,1,1,1,1,1,1,1,1
Beaumond Heights,1,1,1,1,1,1,1,1,1
Humber Summit,1,1,1,1,1,1,1,1,1
Humbergate,1,1,1,1,1,1,1,1,1
Jamestown,1,1,1,1,1,1,1,1,1
Kingsview Village,1,1,1,1,1,1,1,1,1
Martin Grove Gardens,1,1,1,1,1,1,1,1,1
Mount Olive,1,1,1,1,1,1,1,1,1


<H1>Place identified in Cluster4 is 'Westmount'</H1>


<H1>Summary</H1>

After Analysis of Postal codes of Toronto and clustering and identifying number of venues in each neighbourhood , below places are identified for opening a pizza store ,

Where there are more than 20 venues in a neighbourhood we have choses 3 probable places in that cluster

Where there are less than 20 venues in a neighbourhood we have choses only 1 place in that cluster

<H4>Places identified in Cluster0 are 'Union Station' , 'Toronto Islands' ,'Harbourfront East'</H4>

<H4>Places identified in Cluster1 are 'Swansea' , 'High Park' ,'The Junction South' </H4>

<H4>Place identified in Cluster2 is 'Steeles West / L'Amoreaux West' </H4>

<H4>Places identified in Cluster3 are 'Fairview' , 'Oriole' ,'Henry Farm' </H4>

<H4>Place identified in Cluster4 is 'Westmount'</H4>